In [187]:
import requests
from bs4 import BeautifulSoup
import datetime as DT
import pymysql
import threading
import time

In [221]:
mainUrl='https://www.nerjstal.ru/catalog/nerzhaveyka/truba/'
url2='https://www.nerjstal.ru'

In [222]:
class Tube():
    name=None
    marka=None
    diam=None
    thikness=None

In [228]:
ids=[]
resp=requests.get(mainUrl)
pageInfo=BeautifulSoup(resp.content,'lxml')
for pi in pageInfo.findAll('div',class_='catalog-list-item__name tb__cell'):
    ids.append(pi.find('a').get('href'))

for i in range(2,64):
    r=requests.get(mainUrl+'page'+str(i)+'/')
    pi=BeautifulSoup(resp.content,'lxml')
    for pi in pageInfo.findAll('div',class_='catalog-list-item__name tb__cell'):
        ids.append(pi.find('a').get('href'))

In [231]:
def mainParser2(id_):
    r=requests.get(url2+id_)
    tubeInfo=BeautifulSoup(r.content,'lxml')
    t=Tube()
    t.name=tubeInfo.find('div',class_='page-title').text
    t.marka=tubeInfo.find('div',class_='tabs__item tabs__item_visible tabs-item').findAll('div',class_='tb__row')[2].findAll('div',class_='tb__cell')[1].text
    t.diam=tubeInfo.find('div',class_='tabs__item tabs__item_visible tabs-item').findAll('div',class_='tb__row')[3].findAll('div',class_='tb__cell')[1].text
    t.thikness=tubeInfo.find('div',class_='tabs__item tabs__item_visible tabs-item').findAll('div',class_='tb__row')[4].findAll('div',class_='tb__cell')[1].text
    tubes.append(t)
    return t

Один поток

In [232]:
tubes_1=[]
start = time.time()
for i in ids:
    tubes_1.append(mainParser2(i))
print(f'Threading: {time.time() - start : .2f} seconds')

KeyboardInterrupt: 

Многопоточность

In [233]:
tubes_2=[]

def addTiList(a):
    tubes_2.append(mainParser2(a))

start = time.time()
threads = []
for id_ in ids:
    thread = threading.Thread(target=addTiList, args=(id_,))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
print(f'Threading: {time.time() - start : .2f} seconds')

Threading:  5.45 seconds


In [234]:
len(tubes_2)

50

In [235]:
for i in tubes_2:
    print(i.name, i.marka,i.diam,i.thikness)

 Труба нержавеющая AISI 201 □30×15×1мм шлифованная  AISI 201 30×15 1
 Труба нержавеющая квадратная, сталь AISI 201 ☒25×25×1мм зеркало  AISI 201 25×25 1
 Труба нержавеющая квадратная, сталь AISI 201 ☒40×40×1мм шлифованная  AISI 201 40×40 1
 Труба нержавеющая квадратная, сталь AISI 201 ☒20×20×1мм шлифованная  AISI 201 20×20 1
 Труба нержавеющая квадратная, сталь AISI 201 ☒30×30×1мм шлифованная  AISI 201 30×30 1
 Труба нержавеющая квадратная, сталь 08Х18Н10 ☒30×30×1мм матовая  08Х18Н10 30×30 1
 Труба нержавеющая квадратная, сталь 08Х18Н10 ☒20×20×1мм матовая  08Х18Н10 20×20 1
 Труба нержавеющая квадратная, сталь 08Х18Н10 ☒10×10×1мм матовая  08Х18Н10 10×10 1
 Труба нержавеющая квадратная, сталь 08Х18Н10 ☒10×10×1мм зеркало  08Х18Н10 10×10 1
 Труба нержавеющая квадратная, сталь 08Х18Н10 ☒10×10×1мм шлифованная  08Х18Н10 10×10 1
 Труба нержавеющая квадратная, сталь AISI 304 ☒10×10×1мм шлифованная  AISI 304 10×10 1
 Труба нержавеющая квадратная, сталь AISI 201 ☒20×20×1мм зеркало  AISI 201 20×20 

In [32]:
con = pymysql.connect('localhost','root','root','profiles')
cursor = con.cursor()

In [ ]:
createTable='CREATE TABLE tubes ('+\
    'id INT NOT NULL,'+\
    'name VARCHAR(100),'+\
    'marka VARCHAR(100),'+\
    'diametr VARCHAR(100),'+\
    'thikness VARCHAR(100),'+\
    'PRIMARY KEY(id))'

In [ ]:
cursor.execute(createTable)

In [ ]:
insertSQL='INSERT INTO companies (id, name, marka,  diametr, thikness) VALUES (%s, %s, %s, %s, %s)'

In [ ]:
for i in tubes_2:
    cursor.execut(insertSQL,(i.id_,i.name,i.marka,i.diametr,i.thikness))
    con.commit()